### This application will translate text from English into another language

In [ ]:
# !pip install langchain

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

### Using Language Models

In [3]:
GPT_MODEL = "gpt-4"
model = AzureChatOpenAI(
        azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
        api_key=os.environ['OPENAI_API_KEY'],
        openai_api_version=os.environ['OPENAI_API_VERSION'],
        openai_api_type=os.environ['OPENAI_API_TYPE'],
        azure_deployment=GPT_MODEL,
        model=GPT_MODEL,
        temperature=0.7,
        max_tokens=2048
    )

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

response = model.invoke(messages)

In [6]:
print(type(response))

<class 'langchain_core.messages.ai.AIMessage'>


In [10]:
print(response)

content='Ciao!' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-ab6083b5-ee43-4679-aa66-07e0f9a70f7a-0' usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23}


### OutputParsers

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [12]:
parser.invoke(response)

'Ciao!'

### Chaining Different Elements
More commonly, we can "chain" the model with this output parser. This means this output parser will get called every time in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string). We can easily create the chain using the | operator. The | operator is used in LangChain to combine two elements together.

In [13]:
chain = model | parser

In [14]:
chain.invoke(messages)

'ciao!'

### Prompt Templates
PromptTemplates are a concept in LangChain designed to take in raw user input and return data (a prompt) that is ready to pass into a language model. Let's create a PromptTemplate here. It will take in two user variables:

* language: The language to translate text into
* text: The text to translate

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "Translate the following into {language}:"), ("user", "{text}")]
)

In [16]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [17]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

### Chaining together components with LCEL

In [18]:
chain = prompt_template | model | parser

In [19]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'